In [97]:
from classes import Trip
from flights import Flights
from amadeus import Client, ResponseError
from datetime import timedelta, date
import datetime
from hotels1 import hotelFunctions
from transport import transportFunctions
from transport import getNZfromCity
from amadeus import Location
import random

In [98]:
def getLandingDate(initFlight):
    relevantFlight = initFlight[-1]
    return relevantFlight.timeEnd
def getDepartureDate(finFlight):
    relevantFlight = finFlight[0]
    return relevantFlight.timeStart

In [99]:
def chooseCheapestHotel(hotels):
    lowest = 1000000
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost < lowest):
            index = i
            lowest = hotels[i].cost
    return hotels[index]
def chooseMostLuxuriousHotel(hotels):
    highest = 0
    index = 0
    for i in range(0, len(hotels)):
        if (hotels[i].cost > highest):
            index = i
            highest = hotels[i].cost
    return hotels[index]
def chooseHotel(lowCost, luxury, hotelObject, city, passengerCount):
    print(getNZfromCity(city)[0], getNZfromCity(city)[1])
    xCoord = getNZfromCity(city)[0]
    yCoord = getNZfromCity(city)[1]
    hotels = []
    while (hotels == []):
        hotels = hotelObject.getHotelsByGeocode(xCoord, yCoord)
        xCoord += (random.random() - 0.5) * 0.4
        yCoord += (random.random() - 0.5) * 0.4
    print(hotels)
    for i in hotels:
        print(i)
    choosenHotel = None
    if (lowCost):
        choosenHotel = chooseCheapestHotel(hotels)
    elif (luxury):
        choosenHotel = chooseMostLuxuriousHotel(hotels)
    else:
        choosenHotel = random.choice(hotels)
    choosenHotel.cost = choosenHotel.cost * passengerCount
    return choosenHotel

In [100]:
def getTrip(srcAirport, date, duration, passengerCount, isFastPaced, isMuseumOriented, isLuxury, isLowCost, city):
    amadeusBaseObject = Client(
        client_id='CGwOmHn7cmfAIuUcbqUiaPC5LAyAvwAG',
        client_secret='rKvILHDsjxcCh6yq'
    )
    duration -= 1
    flightObject = Flights(amadeusBaseObject)
    nearestAirport = amadeusBaseObject.reference_data.locations.get(
    keyword=city,
    subType=Location.ANY
)
    nearestAirport = nearestAirport.data
    destinationAirport = nearestAirport[0]['iataCode']
    coordsArr = [nearestAirport[0]['geoCode']['latitude'], nearestAirport[0]['geoCode']['longitude']]
    initFlight = flightObject.getFlightObjects(srcAirport, destinationAirport, date, passengerCount)
    landingDate = str(getLandingDate(initFlight)).split(' ')[0]
    dateFormat = '%Y-%m-%d' 
    dateObject = datetime.datetime.strptime(landingDate, dateFormat)
    finFlight = flightObject.getFlightObjects(destinationAirport, srcAirport, (dateObject + timedelta(days=duration)).strftime("%Y-%m-%d"), passengerCount)
    tripStartTime = getLandingDate(initFlight)
    tripEndTime = getDepartureDate(finFlight)
    hotelObject = hotelFunctions(amadeusBaseObject)
    hotel = chooseHotel(isLowCost, isLuxury, hotelObject, city, passengerCount)
    print(hotel)
    attractions = hotelObject.getActivities(hotel.destination.split(',')[0], hotel.destination.split(',')[1], 2)
    print(attractions)

In [96]:
getTrip('MAD', '2022-07-07', 5, 4, True, False, True, False, 'New York')


{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2022-07-07', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT29H55M', 'segments': [{'departure': {'iataCode': 'MAD', 'terminal': '4S', 'at': '2022-07-07T19:45:00'}, 'arrival': {'iataCode': 'CMN', 'terminal': '2', 'at': '2022-07-07T20:35:00'}, 'carrierCode': 'AT', 'number': '971', 'aircraft': {'code': '73H'}, 'operating': {'carrierCode': 'AT'}, 'duration': 'PT2H50M', 'id': '264', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'CMN', 'terminal': '1', 'at': '2022-07-08T15:50:00'}, 'arrival': {'iataCode': 'JFK', 'at': '2022-07-08T18:40:00'}, 'carrierCode': 'AT', 'number': '200', 'aircraft': {'code': '789'}, 'operating': {'carrierCode': 'AT'}, 'duration': 'PT7H50M', 'id': '265', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1500.32', 'base': '484.00', 'fees': [

{'type': 'flight-offer', 'id': '1', 'source': 'GDS', 'instantTicketingRequired': False, 'nonHomogeneous': False, 'oneWay': False, 'lastTicketingDate': '2022-07-11', 'numberOfBookableSeats': 9, 'itineraries': [{'duration': 'PT9H10M', 'segments': [{'departure': {'iataCode': 'EWR', 'at': '2022-07-11T21:50:00'}, 'arrival': {'iataCode': 'LHR', 'at': '2022-07-12T08:45:00'}, 'carrierCode': '6X', 'number': '188', 'aircraft': {'code': '777'}, 'operating': {'carrierCode': '6X'}, 'duration': 'PT5H55M', 'id': '27', 'numberOfStops': 0, 'blacklistedInEU': False}, {'departure': {'iataCode': 'LHR', 'at': '2022-07-12T10:30:00'}, 'arrival': {'iataCode': 'MAD', 'at': '2022-07-12T13:00:00'}, 'carrierCode': '6X', 'number': '9931', 'aircraft': {'code': '320'}, 'operating': {'carrierCode': '6X'}, 'duration': 'PT1H30M', 'id': '28', 'numberOfStops': 0, 'blacklistedInEU': False}]}], 'price': {'currency': 'EUR', 'total': '1281.80', 'base': '1108.00', 'fees': [{'amount': '0.00', 'type': 'SUPPLIER'}, {'amount': '0

40.68295 -73.9708


NameError: name 'random' is not defined

In [101]:
chooseHotel(True, False, hotelFunctions(amadeusBaseObject), 'New York', 4)

40.68295 -73.9708
ERROR IS:  [400]

ERROR IS:  [400]

ERROR IS:  [400]

ERROR IS:  [400]

ERROR IS:  [400]



KeyboardInterrupt: 